In [1]:
import os
import shutil
from tqdm import tqdm

In [2]:
train_folder = "/Users/monicagullapalli/Downloads/emotion_detection/train"
validation_folder = "/Users/monicagullapalli/Downloads/emotion_detection/validation"

In [3]:
combined_folder = "//Users/monicagullapalli/Downloads/Neural-networks-assignments/Neural-Networks-project/combined"

In [4]:
def copy_files_with_labels(source_folder, destination_folder):
    for label in os.listdir(source_folder):
        label_folder = os.path.join(source_folder, label)
        if os.path.isdir(label_folder):
            destination_label_folder = os.path.join(destination_folder, label)
            if not os.path.exists(destination_label_folder):
                os.makedirs(destination_label_folder)
            for file in tqdm(os.listdir(label_folder), desc=f"Copying {label}"):
                shutil.copy(os.path.join(label_folder, file), destination_label_folder)


copy_files_with_labels(train_folder, combined_folder)


copy_files_with_labels(validation_folder, combined_folder)

print("Data combined successfully.")

Copying Happy:   0%|          | 0/7215 [00:00<?, ?it/s]

Copying Disgust: 100%|██████████| 55/55 [00:00<00:00, 2749.87it/s]

Data combined successfully.


In [5]:
import tensorflow as tf
from tensorflow.keras import layers

model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(7, activation='softmax') 
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

from tensorflow.keras.preprocessing.image import ImageDataGenerator


datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(
    '/Users/monicagullapalli/Downloads/emotion_detection/train',
    target_size=(64, 64), 
    batch_size=32,
    class_mode='sparse'  
)


validation_generator = datagen.flow_from_directory(
    '/Users/monicagullapalli/Downloads/emotion_detection/validation',
    target_size=(64, 64), 
    batch_size=32,
    class_mode='sparse' 
)


model.fit(train_generator, validation_data=validation_generator, epochs=10)

Found 28709 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.
Epoch 1/10
898/898 [==============================] - 39s 43ms/step - loss: 1.6115 - accuracy: 0.3754 - val_loss: 1.5726 - val_accuracy: 0.3993
Epoch 2/10
898/898 [==============================] - 41s 45ms/step - loss: 1.4191 - accuracy: 0.4575 - val_loss: 1.5646 - val_accuracy: 0.3984
Epoch 3/10
898/898 [==============================] - 53s 59ms/step - loss: 1.2965 - accuracy: 0.5098 - val_loss: 1.5578 - val_accuracy: 0.4171
Epoch 4/10
898/898 [==============================] - 93s 103ms/step - loss: 1.1745 - accuracy: 0.5608 - val_loss: 1.6054 - val_accuracy: 0.4079
Epoch 5/10
898/898 [==============================] - 71s 79ms/step - loss: 1.0574 - accuracy: 0.6092 - val_loss: 1.6656 - val_accuracy: 0.4171
Epoch 6/10
898/898 [==============================] - 75s 84ms/step - loss: 0.9333 - accuracy: 0.6565 - val_loss: 1.7957 - val_accuracy: 0.4235
Epoch 7/10
898/898 [==============================

In [6]:
model.save('/Users/monicagullapalli/Downloads/Neural-networks-assignments/Neural-Networks-project/final_model.h5')

/Users/monicagullapalli/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [8]:
from PIL import Image, ImageTk

In [16]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import webbrowser

# Load the pre-trained emotion classification model
model = load_model('/Users/monicagullapalli/Downloads/Neural-networks-assignments/Neural-Networks-project/final_model.h5')

class_labels = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Neutral', 5: 'Sad', 6: 'Surprise'}

# Define Spotify playlist URLs for each emotion
spotify_playlists = {
    'Angry': 'https://spotify.com/playlist/angry_playlist',
    'Sad': 'https://spotify.com/playlist/sad_playlist',
    'Happy': 'https://spotify.com/playlist/happy_playlist'
}

# Initialize the camera
cap = cv2.VideoCapture(0)

# Function to capture image and perform actions
def capture_image():
    ret, frame = cap.read()
    processed_frame = cv2.resize(frame, (64, 64))
    processed_frame = processed_frame / 255.0

    input_data = np.expand_dims(processed_frame, axis=0)

    predictions = model.predict(input_data)
    predicted_class = np.argmax(predictions)
    predicted_label = class_labels[predicted_class]

    # Perform actions based on the predicted class label
    if predicted_label == 'Angry':
        # Redirect to Angry playlist on Spotify
        webbrowser.open(spotify_playlists['Angry'])

    elif predicted_label == 'Sad':
        # Redirect to Sad playlist on Spotify
        webbrowser.open(spotify_playlists['Sad'])

    elif predicted_label == 'Happy':
        # Redirect to Happy playlist on Spotify
        webbrowser.open(spotify_playlists['Happy'])

# Create a Tkinter window
window = tk.Tk()

# Function to capture frame and perform actions
def capture_frame():
    # Capture a frame from the camera
    ret, frame = cap.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(frame)

    # Display the captured frame
    image.show()

    # Perform actions on the captured frame
    capture_image()

# Create a capture button
capture_button = tk.Button(window, text='Capture', command=capture_frame)
capture_button.pack()

# Start the Tkinter event loop
window.mainloop()

# Release the camera
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 106ms/step


KeyboardInterrupt: 

: 